# The Product Pricer Continued

A model that can estimate how much something costs, from its description.

## AT LAST - it's time for Fine Tuning!

After all this data preparation, and old school machine learning, we've finally arrived at the moment you've been waiting for. Fine-tuning a model.

In [1]:
# imports

import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
from itemInfo import Item
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI
from anthropic import Anthropic

In [2]:
# moved our Tester into a separate package
# call it with Tester.test(function_name, test_dataset)

from runTest import Tester

In [3]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [4]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
openai = OpenAI()

In [6]:
%matplotlib inline

In [7]:
# Let's avoid curating all our data again! Load in the pickle files:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [8]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 200 examples (and 1 epoch)

fine_tune_train = train[:200]
fine_tune_validation = train[200:250]

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [9]:
# First let's work on a good prompt for a Frontier model
# Notice that I'm removing the " to the nearest dollar"
# When we train our own models, we'll need to make the problem as easy as possible, 
# but a Frontier model needs no such simplification.

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

In [10]:
messages_for(train[0])

[{'role': 'system',
  'content': 'You estimate prices of items. Reply only with the price, no explanation'},
 {'role': 'user',
  'content': 'How much does this cost?\n\nBBOUNDER 16 Pack Linkable LED Utility Shop Light, 4 FT, 6500K Cool Daylight, 4400 LM, 48 Inch Integrated Fixture for Garage, 40W Equivalent 250W, Surface + Suspension Mount, White\nSuper brightness With 4400 lumens brightness at 6500K cool daylight white, these 4ft led shop lights only consume 40W while provide the bright illumination with 250W (6 times more than conventional fluorescent lights). It is an amazing LED Efficiency of 110 lm/w, which can'},
 {'role': 'assistant', 'content': 'Price is $194.99'}]

In [11]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [12]:
print(make_jsonl(train[:3]))

{"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nBBOUNDER 16 Pack Linkable LED Utility Shop Light, 4 FT, 6500K Cool Daylight, 4400 LM, 48 Inch Integrated Fixture for Garage, 40W Equivalent 250W, Surface + Suspension Mount, White\nSuper brightness With 4400 lumens brightness at 6500K cool daylight white, these 4ft led shop lights only consume 40W while provide the bright illumination with 250W (6 times more than conventional fluorescent lights). It is an amazing LED Efficiency of 110 lm/w, which can"}, {"role": "assistant", "content": "Price is $194.99"}]}
{"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nNT FAIRING Glossy White Red Injection Mold Fairing Fit for Yamaha 2009 2010 2011 YZF R1 R1000 YZF-R1 New Painted Kit ABS Plastic Moto

In [13]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [14]:
write_jsonl(fine_tune_train, "fine_tune_train.jsonl")

In [15]:
write_jsonl(fine_tune_validation, "fine_tune_validation.jsonl")

In [16]:
with open("fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [17]:
train_file

FileObject(id='file-FYjZ4oXQa2yZx1yRqJ7E7p', bytes=155785, created_at=1740318732, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [18]:
with open("fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [19]:
validation_file

FileObject(id='file-K2m9vrFium4fGVpkKY8MP9', bytes=39671, created_at=1740318740, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

# Step 2

I love Weights and Biases - a beautiful, free platform for monitoring training runs.  
Weights and Biases is integrated with OpenAI for fine-tuning.

First set up your weights & biases free account at:

https://wandb.ai

From the Avatar >> Settings menu, near the bottom, you can create an API key.

Then visit the OpenAI dashboard at:

https://platform.openai.com/account/organization

In the integrations section, you can add your Weights & Biases key.

## And now time to Fine-tune!

In [23]:
wandb_integration = {"type": "wandb", "wandb": {"project": "gpt-pricer"}}

In [24]:
train_file.id

'file-FYjZ4oXQa2yZx1yRqJ7E7p'

In [26]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    hyperparameters={"n_epochs": 1},
    integrations = [wandb_integration],
    suffix="pricer"
)

FineTuningJob(id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC', created_at=1740319201, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-VMADVZuXtcBJfUbLWZQNHn2u', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-FYjZ4oXQa2yZx1yRqJ7E7p', validation_file='file-K2m9vrFium4fGVpkKY8MP9', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC'))], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='pricer')

In [27]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC', created_at=1740319201, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-VMADVZuXtcBJfUbLWZQNHn2u', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-FYjZ4oXQa2yZx1yRqJ7E7p', validation_file='file-K2m9vrFium4fGVpkKY8MP9', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC'))], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix

In [28]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [29]:
job_id

'ftjob-VCg0rUTPRddUlD0q1p6UxCgC'

In [30]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC', created_at=1740319201, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-VMADVZuXtcBJfUbLWZQNHn2u', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-FYjZ4oXQa2yZx1yRqJ7E7p', validation_file='file-K2m9vrFium4fGVpkKY8MP9', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-VCg0rUTPRddUlD0q1p6UxCgC'))], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='pricer')

In [31]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-3FUIO9kIOhua2dYA4vu9gAz0', created_at=1740319201, level='info', message='Validating training file: file-FYjZ4oXQa2yZx1yRqJ7E7p and validation file: file-K2m9vrFium4fGVpkKY8MP9', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-Iu1SUPlP7LSSpJULaaVw9DSH', created_at=1740319201, level='info', message='Created fine-tuning job: ftjob-VCg0rUTPRddUlD0q1p6UxCgC', object='fine_tuning.job.event', data={}, type='message')]

# Step 3

Test our fine tuned model

In [32]:
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [33]:
fine_tuned_model_name

In [34]:
# The prompt

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [35]:
# Try this out

messages_for(test[0])

[{'role': 'system',
  'content': 'You estimate prices of items. Reply only with the price, no explanation'},
 {'role': 'user',
  'content': 'How much does this cost?\n\nBase Antenna, 1.25m, 4ft w/Ground Plane\nLook at all you get at an incredible price!You get a 300 watt 5/8 wave ground plane base antenna for 220 MHz that gives you the maximum possible calculated gain of any single element antenna. Other 5/8 waves cant work any better -- no matter how much they cost. You get shunt fed matching network for lowest possible SWR over entire 2 meter band. Plus, it bleeds off unwanted static. You get MFJs RapidTuneRadiator for quick accurate tuning. Easy installation to any 1 to 1 1/2 inch mast Strong light'},
 {'role': 'assistant', 'content': 'Price is $'}]

In [36]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("The price is roughly $99.99 because blah blah")

In [37]:
# The function for gpt-4o-mini

def gpt_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name, 
        messages=messages_for(item),
        seed=42,
        max_tokens=7
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [39]:

fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print("Fine-tuned model name:", fine_tuned_model_name)


Fine-tuned model name: None


In [40]:
print(test[0].test_prompt())

How much does this cost to the nearest dollar?

Base Antenna, 1.25m, 4ft w/Ground Plane
Look at all you get at an incredible price!You get a 300 watt 5/8 wave ground plane base antenna for 220 MHz that gives you the maximum possible calculated gain of any single element antenna. Other 5/8 waves cant work any better -- no matter how much they cost. You get shunt fed matching network for lowest possible SWR over entire 2 meter band. Plus, it bleeds off unwanted static. You get MFJs RapidTuneRadiator for quick accurate tuning. Easy installation to any 1 to 1 1/2 inch mast Strong light

Price is $


In [43]:
Tester.test(gpt_fine_tuned, test)